In [134]:
import pandas as pd

In [144]:
df = pd.read_csv("../processed-datasets/external-feature-dataset.csv")

In [145]:
df = df.drop(["Unnamed: 0", "domain"], axis=1)

df.head()

,class,url,status_code,html
0,0,https://crowdyfan.com/,200,b'eNrtfet220bS4G/rHL9Dh5kxpS8CCYB3ymI+WZJtzcS2...
1,0,https://dreamaways.com/,403,b'eNqzySjJzbHj5bLJSE1MsbMpySzJSbUzMTBWcMsvSspM...
2,0,https://www.worldtravelserver.com/,200,b'eNrsvXuX4zaSL/h396fgpI+PK68lFfWWUq7ccdvt7t6e...
3,0,https://www.baarty.com/,200,b'eNrtfdtyG0ey4LMVoX8oY+KMpDho3HiXSXgoirY5K1k8...
4,0,https://ladamotors63.ru/,200,b'eNrtPf1zG0WWP1t/RWfudmMdHo0+bMtyLLMkhKvUZoFa...


In [146]:
df.groupby('status_code').count()

,class,url,html
status_code,,,
200,52531,52531,52531
201,4,4,4
202,2,2,2
203,3,3,3
204,363,363,363
301,1,1,1
302,2,2,2
400,1029,1029,1029
401,112,112,112


In [147]:
df.groupby('class').count()

,url,status_code,html
class,,,
0,43609,43609,43609
1,30589,30589,30589


### Extract metadata and page title from page information

In [148]:
from multiprocessing import freeze_support, Pool, Value
import zlib
import base64
import sys
from bs4 import BeautifulSoup

count = 0

metainfo = ["keywords", "description", "og_description", "og:description", "og_title", "og:title"]

def extractor(row):
    str_compressed = row["html"]
    global count
    
    metadata = {}
    
    soup = BeautifulSoup(zlib.decompress(base64.b64decode(str_compressed[2:-1])), "html.parser")
    
    
    title = soup.head.title.text if soup.head and soup.head.title else "None"
    
    for item in metadata:
        metadata[item] = soup.find("meta", property=item)
    
    for item in metadata:
        if metadata[item] != None:
            if "content" in metadata[item]:
                metadata[item] = metadata[item]["content"] 
            else:
                metadata[item] =""
        else:
            metadata[item] =""
    
    metadata["title"] = title
    
    count += 1
    
    print("Status:", count , end="\r", flush=True)
    
    str_metadata = ""
    
    for item in metadata:
        str_metadata += metadata[item] + " "
    
    return str_metadata

In [149]:
tdf = df

In [150]:
tdf["meta_info"] = tdf.apply(extractor, axis = 1, result_type ='expand')

/home/yk/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


/home/yk/.local/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [158]:
tdf.head()

,class,url,status_code,meta_info
0,0,https://crowdyfan.com/,200,Crowdyfan Sports News Nutrition More
1,0,https://dreamaways.com/,403,403 Forbidden
2,0,https://www.worldtravelserver.com/,200,London travel agencies United Kingdom weather ...
3,0,https://www.baarty.com/,200,Soluciones Para Bares Baartycom
4,0,https://ladamotors63.ru/,200,Lada Motors


In [152]:
tdf.isna()

,class,url,status_code,meta_info
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
74193,False,False,False,False
74194,False,False,False,False
74195,False,False,False,False
74196,False,False,False,False


In [151]:
tdf.drop("html", axis=1, inplace=True)

In [153]:
tdf.head()

,class,url,status_code,meta_info
0,0,https://crowdyfan.com/,200,"Crowdyfan | Sports News, Nutrition & More"
1,0,https://dreamaways.com/,403,403 Forbidden
2,0,https://www.worldtravelserver.com/,200,"London, travel agencies, United Kingdom, weath..."
3,0,https://www.baarty.com/,200,Soluciones Para Bares - Baarty.com
4,0,https://ladamotors63.ru/,200,Lada Motors


In [154]:
import re

In [155]:
df['meta_info'] = df['meta_info'].apply(lambda x: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", x)) 

In [156]:
df.head()

,class,url,status_code,meta_info
0,0,https://crowdyfan.com/,200,Crowdyfan Sports News Nutrition More
1,0,https://dreamaways.com/,403,403 Forbidden
2,0,https://www.worldtravelserver.com/,200,London travel agencies United Kingdom weather ...
3,0,https://www.baarty.com/,200,Soluciones Para Bares Baartycom
4,0,https://ladamotors63.ru/,200,Lada Motors


In [159]:
tdf.to_csv("../processed-datasets/metainfo_decompressed.csv", sep="$", index=False)